In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import itertools
import cv2
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [19]:
pwd

'/workspace/aman/MANYA'

In [20]:
df = pd.read_csv('GroundTruth.csv')
df.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
0,ISIC_0024306,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0024307,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0024308,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0024309,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0024310,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df.shape

(10015, 8)

In [22]:
dfDF = df[df["DF"]==1]

In [23]:
dfDF.head()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
12,ISIC_0024318,0.0,0.0,0.0,0.0,0.0,1.0,0.0
24,ISIC_0024330,0.0,0.0,0.0,0.0,0.0,1.0,0.0
80,ISIC_0024386,0.0,0.0,0.0,0.0,0.0,1.0,0.0
90,ISIC_0024396,0.0,0.0,0.0,0.0,0.0,1.0,0.0
247,ISIC_0024553,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
dfDF.drop(columns=["MEL", "NV", "AKIEC", "BCC", "BKL", "VASC"], inplace=True)

<ipython-input-24-50a669caf44d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfDF.drop(columns=["MEL", "NV", "AKIEC", "BCC", "BKL", "VASC"], inplace=True)


In [25]:
dfDF.head()

,image,DF
12,ISIC_0024318,1.0
24,ISIC_0024330,1.0
80,ISIC_0024386,1.0
90,ISIC_0024396,1.0
247,ISIC_0024553,1.0


In [26]:
dfNotDF = df[df["DF"]==0]

In [27]:
dfNotDF.tail()

,image,MEL,NV,BCC,AKIEC,BKL,DF,VASC
10010,ISIC_0034316,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10011,ISIC_0034317,1.0,0.0,0.0,0.0,0.0,0.0,0.0
10012,ISIC_0034318,0.0,0.0,0.0,0.0,1.0,0.0,0.0
10013,ISIC_0034319,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10014,ISIC_0034320,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [28]:
dfNotDF.drop(columns=["MEL", "NV", "AKIEC", "BCC", "BKL", "VASC"], inplace=True)

<ipython-input-28-819011272c45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNotDF.drop(columns=["MEL", "NV", "AKIEC", "BCC", "BKL", "VASC"], inplace=True)


In [29]:
dfNotDF.head()

,image,DF
0,ISIC_0024306,0.0
1,ISIC_0024307,0.0
2,ISIC_0024308,0.0
3,ISIC_0024309,0.0
4,ISIC_0024310,0.0


In [30]:
dfNotDF.shape

(9900, 2)

In [31]:
dfDF.shape

(115, 2)

In [32]:
dfNotDF = dfNotDF.sample(frac = 0.02, random_state = 40)

In [33]:
dfDF.shape

(115, 2)

In [34]:
dfNotDF.shape

(198, 2)

In [35]:
df = pd.concat([dfDF, dfNotDF])

In [36]:
df.head()

,image,DF
12,ISIC_0024318,1.0
24,ISIC_0024330,1.0
80,ISIC_0024386,1.0
90,ISIC_0024396,1.0
247,ISIC_0024553,1.0


In [37]:
df.shape

(313, 2)

In [38]:
df = df.sample(frac=1, random_state=30)

In [39]:
df.head()

,image,DF
6696,ISIC_0031002,1.0
6788,ISIC_0031094,0.0
7066,ISIC_0031372,1.0
6878,ISIC_0031184,0.0
3005,ISIC_0027311,0.0


In [40]:
imageArray = df["image"].to_numpy()

In [41]:
imageArray.shape

(313,)

In [42]:
pwd

'/workspace/aman/MANYA'

In [43]:
cd images

/workspace/aman/MANYA/images


In [44]:
pwd

'/workspace/aman/MANYA/images'

In [45]:
Images = []

In [46]:
def processImage(image):
    image = cv2.resize(image, (150, 200))
    image = image.astype("float32") / 255
    return image

In [47]:
for img in imageArray:
    path = img + ".jpg"
    image = cv2.imread(path)
    image = processImage(image)
    Images.append(image)

In [48]:
len(Images)

313

In [49]:
pwd

'/workspace/aman/MANYA/images'

In [50]:
np.save("DFIMAGES.npy", Images)

In [51]:
output = df["DF"].to_numpy()

In [52]:
len(output)

313

In [53]:
output.shape

(313,)

In [54]:
xTrain, xTest, yTrain, yTest = train_test_split(np.array(Images), output, test_size = 0.2, random_state = 30)

In [55]:
xTrain.shape

(250, 200, 150, 3)

In [56]:
yTrain.shape

(250,)

In [57]:
xTrain, xVal, yTrain, yVal = train_test_split(xTrain, yTrain, test_size = 0.2, random_state = 30)

In [58]:
xVal.shape

(50, 200, 150, 3)

In [59]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(200, 150, 3)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 150, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 150, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 75, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 100, 75, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 75, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 75, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 37, 64)        0

In [60]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["Accuracy"])

In [61]:
history = model.fit(xTrain, yTrain, validation_data = (xVal, yVal), epochs = 10, batch_size = 32)

Epoch 1/10
7/7 [==============================] - 5s 378ms/step - loss: 2.1780 - accuracy: 0.0000e+00 - val_loss: 0.7027 - val_accuracy: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 1s 106ms/step - loss: 0.6850 - accuracy: 0.0000e+00 - val_loss: 0.6828 - val_accuracy: 0.0000e+00
Epoch 3/10
7/7 [==============================] - 1s 80ms/step - loss: 0.6503 - accuracy: 0.0000e+00 - val_loss: 0.6933 - val_accuracy: 0.0000e+00
Epoch 4/10
7/7 [==============================] - 0s 52ms/step - loss: 0.6290 - accuracy: 0.0000e+00 - val_loss: 0.6828 - val_accuracy: 0.0000e+00
Epoch 5/10
7/7 [==============================] - 0s 47ms/step - loss: 0.6207 - accuracy: 0.0000e+00 - val_loss: 0.6693 - val_accuracy: 0.0000e+00
Epoch 6/10
7/7 [==============================] - 0s 46ms/step - loss: 0.6387 - accuracy: 0.0000e+00 - val_loss: 0.6352 - val_accuracy: 0.0000e+00
Epoch 7/10
7/7 [==============================] - 0s 42ms/step - loss: 0.6021 - accuracy: 0.0000e+00 - val_loss: 0.6

In [62]:
predictions = model.predict(xTest)

In [63]:
preds = []

In [64]:
for p in predictions:
    if p>=0.5:
        preds.append(1)
    else:
        preds.append(0)

In [65]:
report = classification_report(yTest, preds, target_names = ["NOT NV", "NV"])

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [66]:
print(report)

              precision    recall  f1-score   support

      NOT NV       0.62      1.00      0.76        39
          NV       0.00      0.00      0.00        24

    accuracy                           0.62        63
   macro avg       0.31      0.50      0.38        63
weighted avg       0.38      0.62      0.47        63

